In [1]:
import re
import os
import pandas as pd
from io import StringIO

In [2]:
def latex_tree_table(f_name):

    with open(f_name,"r") as f:
        data = f.read()

    data = re.sub(r"[\n]+","\n",data)
    data = data[:data.index("number of visited networks")]
    data = data.strip("\n")
    data = [ln.split() for ln in data.split("\n")]
    data = \
    [[wd.replace(ln[0],"") if wd.replace("!","") == ln[0] and i!=0 else wd for i,wd in enumerate(ln)] for ln in data]
    data = [[wd.replace("032_","").replace("_001","").replace("_",",") for wd in ln] for ln in data]
    data = [[re.sub(r"^0","", wd).replace(",0",",").replace("!0","!") for wd in ln] for ln in data]
    data = [[re.sub(r"[!]+",r"\\bf ",wd) for wd in ln] for ln in data]
    data = "\\\\\n".join([" & ".join(ln) for ln in data])
    data = data + "\\\\"
    data = data.replace("16b","16").replace("8b","8").replace("32b","32")
    
    print(data)

In [3]:
def latex_tree_tables():

    base_path = "/home/lucas/Documents/perceptronac/complexity/scripts/glch_experiments/glch_results/"
    for method in ["gift_wrapping","tie_break","angle_rule"]:
        for title in ["rate_vs_params","rate_vs_energy_noisy","model_bits_vs_data_bits"]:
            for constraint in ["unconstrained","constrained"]:
                if (method != "tie_break") or (constraint != "unconstrained"):
                    latex_tree_table(os.path.join(base_path,f"glch2D_{method}_{constraint}_{title}_tree.txt"))
                    print("\n")


In [4]:
def latex_ref_node_table(f_name,lmbda,complexity):

    with open(f_name,"r") as f:
        data = f.read()

    data = data[data.index("reference nodes:"):]
    data = data[data.index("labels"):]
    data = "\n".join(data.split("\n")[1:])

    df = pd.read_csv(StringIO(re.sub(" +"," ",data)), sep=" ",header=None,dtype=str)

    lmbda_str = f"L{lmbda}"

    multiplication_factor = (1e10) if complexity == "flops" else (1e6)
    
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: x.replace(lmbda_str,"").replace("D","").replace("N",",").replace("M",","))

    df.iloc[:,1] = df.iloc[:,1].apply(lambda x: str(float(x)/multiplication_factor))

    df.iloc[:,1] = df.iloc[:,1].apply(lambda x: '{0:.3f}'.format(float(x)))
    
    df.iloc[:,2] = df.iloc[:,2].apply(lambda x: '{0:.3f}'.format(float(x)))
    
    table_str = " \\\\\n".join([" & ".join(ln) for ln in df.values.tolist()])

    table_str = table_str + " \\\\\n"
    
    print(table_str)



In [5]:
def latex_ref_node_tables():
    
    base_path = "/home/lucas/Documents/perceptronac/complexity/scripts/glch_experiments/glch_results/"
    
    for method in ["gift_wrapping","tie_break","angle_rule"]:
        for constraint in ["unconstrained","constrained"]:
            for lmbda in ["5e-3","1e-2","2e-2"]:
                for complexity in ["params","flops"]:
                    if (method != "tie_break") or (constraint != "unconstrained"):
                        print(method,constraint,lmbda,complexity,"\n")
                        latex_ref_node_table(
                            os.path.join(base_path,f"glch2D_{method}_{constraint}_lambdas_{lmbda}_{complexity}_vs_loss_start_left_hulls.txt"),
                            lmbda,complexity
                        )

In [6]:
# latex_tree_tables()

In [8]:
# latex_ref_node_tables()